In [1]:
# imports
import pandas as pd

## Indicators

In [2]:
# first of all we deserialize our dataframe
df = pd.read_csv("../dataset/customer_supermarket_sale_qta_merge.csv")
print(df.head())

   Unnamed: 0  BasketID           BasketDate  ProdID  Qta  Sale  CustomerID  \
0           0    536365  2010-01-12 08:26:00   21730    6  4.25       17850   
1           1    536365  2010-01-12 08:26:00   22752    2  7.65       17850   
2           2    536365  2010-01-12 08:26:00   71053    6  3.39       17850   
3           3    536365  2010-01-12 08:26:00  84029E    6  3.39       17850   
4           4    536365  2010-01-12 08:26:00  84029G    6  3.39       17850   

  CustomerCountry                            ProdDescr  
0  United Kingdom    GLASS STAR FROSTED T-LIGHT HOLDER  
1  United Kingdom         SET 7 BABUSHKA NESTING BOXES  
2  United Kingdom                  WHITE METAL LANTERN  
3  United Kingdom       RED WOOLLY HOTTIE WHITE HEART.  
4  United Kingdom  KNITTED UNION FLAG HOT WATER BOTTLE  


# I - indicator
the total number of items purchased by a customer during the period of
observation.

In [51]:
df_I = df.groupby("CustomerID").sum().reset_index()
df_I = df_I[["CustomerID", "Qta"]]
df_I

,CustomerID,Qta
0,12346,0
1,12347,2458
2,12348,2341
3,12349,631
4,12350,197
...,...,...
4367,18280,45
4368,18281,54
4369,18282,98
4370,18283,1357


In [6]:
#!! look at this, it might be an outlier
print(df[df.CustomerID == 12346])

       Unnamed: 0  BasketID           BasketDate ProdID    Qta  Sale  \
37034       37034    541431  2011-01-18 10:01:00  23166  74215  1.04   
37039       37039    541433  2011-01-18 10:17:00  23166 -74215  1.04   

       CustomerID CustomerCountry                       ProdDescr  
37034       12346  United Kingdom  MEDIUM CERAMIC TOP STORAGE JAR  
37039       12346  United Kingdom  MEDIUM CERAMIC TOP STORAGE JAR  


# Iu - indicator
the number of distinct items bought by a customer in the period of
observation.

In [14]:
df_iu = df.groupby('CustomerID')['ProdID'].nunique()
df_iu

CustomerID
12346      1
12347    103
12348     22
12349     73
12350     17
        ... 
18280     10
18281      7
18282     12
18283    263
18287     59
Name: ProdID, Length: 4372, dtype: int64

# Imax - indicator
the maximum number of items purchased by a customer during a
shopping session

In [48]:
df_imax = df.groupby(["CustomerID", "BasketID"]).Qta.sum()
df_imax = df_imax.groupby(level=0).head(1)

#df_imax = df_imax.max(level=0)

df_imax

CustomerID  BasketID
12346       541431      74215
12347       537626        319
12348       539318       1254
12349       577609        631
12350       543037        197
                        ...  
18280       545712         45
18281       556464         54
18282       562525         75
18283       540350         61
18287       554065        488
Name: Qta, Length: 4372, dtype: int64

### 